Load Urls

In [1]:
import requests
from bs4 import BeautifulSoup

# Sites scraped from
url1 = 'https://www.cnn.com/'
url2 = 'https://www.bbc.com/'
url3 = 'https://www.nytimes.com/'

# Make HTML requests
r1 = requests.get(url1)
r2 = requests.get(url2)
r3 = requests.get(url3)

cnn = r1.content
bbc = r2.content
nyt = r3.content

# Create BeautifulSoup objects for each site
soupCnn = BeautifulSoup(cnn, 'html5lib')
soupBbc = BeautifulSoup(bbc, 'html5lib')
soupNyt = BeautifulSoup(nyt, 'html5lib')

# List of filtered URLs
allFilteredUrls = []

Filter CNN Articles

In [2]:
# Create string version of soup object
soup_str = str(soupCnn)
articleListStr = soup_str[soup_str.index("articleList"):]
urls = []

# Grab all urls from article list
while ( articleListStr.find("\"uri\":") != -1 and articleListStr.find("\"headline\"") != -1 ):
    # Get url
    startIdx = articleListStr.index("\"uri\":")
    endIdx = articleListStr.index("\"headline\"")
    startOffset = 7
    endOffset = -2
    uri = 'https://www.cnn.com' + articleListStr[(startIdx+startOffset) : (endIdx+endOffset)]
    
    # Update position in string for parsing
    articleListStr = articleListStr[endIdx+2:]
    
    # Get headline of article
    title = str(BeautifulSoup(requests.get(uri).content, 'html5lib').find_all('title'))
    startIdx = title.index(">")
    endIdx = title.index("</title>")
    startOffset = 1
    endOffset = 0
    headline = title[startIdx+startOffset:endIdx+endOffset]
    
    # Append headline and url
    urls.append((headline,uri))
    

In [3]:
# Filter urls
noncovidUrls = []
covidStrs = ['pandemic','corona','virus','sick','trump','mask','quarantine','lockdown','home','covid']

# Adds appropriate urls to filtered list
for (h,u) in urls:
    if not any(x in u or x in h for x in covidStrs):
        allFilteredUrls.append((h,u))

Filter BBC Articles

In [4]:
# Gets all BBC links from homepage
medialinks = soupBbc.findAll("a", {"class": "media__link"})

In [5]:
# Filter urls
covidStrs = ['pandemic','corona','virus','sick','trump','mask','quarantine','lockdown','home','covid','utm']

for u in medialinks:
    # Check if filtered words in link name
    u = str(u)
    if any(x in u for x in covidStrs):
        continue     
        
    # Get url of link
    startIdx = u.index('href')
    startOffset = 6
    endIdx = u.index('rev')
    endOffset = -2
    s = startIdx + startOffset
    f = endIdx + endOffset
    url = u[s : f]
    if "http" not in url:
        url = 'https://www.bbc.com' + url
    
    # Check the title of article
    title = str(BeautifulSoup(requests.get(url).content, 'html5lib').find_all('title'))
    startIdx = title.index(">")
    endIdx = title.index("</title>")
    startOffset = 1
    endOffset = 0
    headline = title[startIdx+startOffset:endIdx+endOffset]
    
    # Check if filtered words in headline
    if any(x in headline for x in covidStrs):
        continue

    # Add url to filtered list
    allFilteredUrls.append((headline,url))

Filter NYTimes Articles

In [6]:
# Grab all urls from h2 headings
soup_str = str(soupNyt)
h2Str = soup_str[soup_str.index("h2 class"):]
urls = []

while ( h2Str.find('h2 class') != -1 ):
    # Get next url
    startIdx = h2Str.index('a href')
    startOffset = 8
    h2Str = h2Str[startIdx+startOffset:]
    endIdx = h2Str.index("\">")
    s = startIdx+startOffset
    f = endIdx
    url = h2Str[ : (endIdx)]
    if "http" not in url:
        url = 'https://www.nytimes.com' + url
    h2Str = h2Str[endIdx+2:]
    
    # Check url for filtered words
    if any(x in url for x in covidStrs):
        continue
    
    # Check the title of article
    title = str(BeautifulSoup(requests.get(uri).content, 'html5lib').find_all('title'))
    if (title.find('>') == -1 ):
        continue
    startIdx = title.index(">")
    endIdx = title.index("</title>")
    startOffset = 1
    endOffset = 0
    headline = title[startIdx+startOffset:endIdx+endOffset]
    
    # Makes sure no duplicates added
    if (headline,url) not in urls:
        urls.append((headline,url))

In [7]:
# Filter urls
noncovidUrls = []
covidStrs = ['#commentsContainer','=','pandemic','corona','virus','sick','trump','mask','quarantine','lockdown','home','covid']

for (h,u) in urls:
    if not any(x in u or h in u for x in covidStrs):
        #print(u)
        allFilteredUrls.append((h,u))

In [8]:
# Print out all filtered links along with headline
for title,url in allFilteredUrls:
    print(title)
    print(url)
    print()

March 23 Stocks close in the red despite Fed's rescue efforts - CNN
https://www.cnn.com/business/live-news/stock-market-news-today-032320/index.html

PG&amp;E to plead guilty to 85 counts in 2018 Camp Fire, court filings show - CNN
https://www.cnn.com/2020/03/23/us/pge-guilty-camp-fire-california/index.html

Asian markets and US stock futures are up after another wild day on Wall Street  - CNN
https://www.cnn.com/2020/03/23/investing/global-stocks-monday/index.html

Instacart plans to hire 300,000 more workers as demand surges for grocery deliveries - CNN
https://www.cnn.com/2020/03/23/tech/instacart-hiring/index.html

Like humans, most female mammals live longer than males  - CNN
https://www.cnn.com/2020/03/23/world/mammals-females-live-longer-scn/index.html

Yale's popular 'happiness' course is free online - CNN
https://www.cnn.com/2020/03/23/health/yale-happiness-course-wellness/index.html

Zookeeper's dance routine delights online viewers - CNN Video
https://www.cnn.com/videos/trav